In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
import time


In [23]:
# Data Loading
class AnimalDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.image_files = []
        self.labels = []

        # Load all image file paths and their labels
        for label_idx, label in enumerate(sorted(os.listdir(directory))):
            subdir = os.path.join(directory, label)
            if os.path.isdir(subdir):
                for file in os.listdir(subdir):
                    if file.lower().endswith('.jpg') or file.lower().endswith('.jpeg'):
                        self.image_files.append(os.path.join(subdir, file))
                        self.labels.append(label_idx)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_path = self.image_files[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [24]:
# Data Preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset and split
full_dataset = AnimalDataset('/content/drive/MyDrive/JPEGImages', transform=transform)
train_idx, test_idx = train_test_split(range(len(full_dataset)), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(full_dataset, train_idx)
test_dataset = torch.utils.data.Subset(full_dataset, test_idx)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=8)

In [25]:
# Model Definition
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 50)


In [26]:
# Model Training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(num_epochs):
    model.train()
    for epoch in range(num_epochs):
        start_time = time.time()
        running_loss = 0.0
        for i, (images, labels) in enumerate(tqdm(train_loader)):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Print every 20 steps
            if (i + 1) % 20 == 0:
                print(f'Epoch {epoch+1}, Step {i+1}, Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

        end_time = time.time()
        epoch_time = end_time - start_time
        print(f'Epoch {epoch+1} completed in {epoch_time:.2f} sec')



In [27]:
# Model Evaluation
def evaluate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

# Performance Optimization: You might consider adjusting these during hyperparameter tuning
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
torch.backends.cudnn.benchmark = True

In [28]:
train_model(10)
evaluate_model()

 16%|█▌        | 20/129 [00:11<00:36,  2.99it/s]

Epoch 1, Step 20, Loss: 0.5919


 31%|███       | 40/129 [00:17<00:29,  3.01it/s]

Epoch 1, Step 40, Loss: 0.3277


 47%|████▋     | 60/129 [00:24<00:22,  3.03it/s]

...

Epoch 10, Step 120, Loss: 0.0135


100%|██████████| 129/129 [00:44<00:00,  2.91it/s]


Epoch 10 completed in 44.36 sec


100%|██████████| 33/33 [00:07<00:00,  4.19it/s]

Accuracy: 87.22680913064595%
